In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install optuna
!pip install catboost==0.23

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
from sklearn import datasets, ensemble
import optuna
from catboost import CatBoostRegressor
from tqdm import tqdm

In [ ]:
path = '/content/drive/MyDrive/DACON/lg_am/'

train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
submission = pd.read_csv(path + 'sample_submission.csv')

In [ ]:
target_list = ['Y_01', 'Y_02', 'Y_03', 'Y_04', 'Y_05', 'Y_06',
       'Y_07', 'Y_08', 'Y_09', 'Y_10', 'Y_11', 'Y_12', 'Y_13', 'Y_14']

In [ ]:
for i in tqdm(range(len(target_list))):
    globals()['train_{}'.format(target_list[i])] = train[['X_01', 'X_02', 'X_03', 'X_04', 'X_05', 'X_06', 'X_07', 'X_08',
       'X_09', 'X_10', 'X_11', 'X_12', 'X_13', 'X_14', 'X_15', 'X_16', 'X_17',
       'X_18', 'X_19', 'X_20', 'X_21', 'X_22', 'X_23', 'X_24', 'X_25', 'X_26',
       'X_27', 'X_28', 'X_29', 'X_30', 'X_31', 'X_32', 'X_33', 'X_34', 'X_35',
       'X_36', 'X_37', 'X_38', 'X_39', 'X_40', 'X_41', 'X_42', 'X_43', 'X_44',
       'X_45', 'X_46', 'X_47', 'X_48', 'X_49', 'X_50', 'X_51', 'X_52', 'X_53',
       'X_54', 'X_55', 'X_56', f'{target_list[i]}']]
    locals()['train_{}'.format(target_list[i])].drop(columns=['X_48', 'X_47', 'X_23', 'X_04'], inplace=True)

  0%|          | 0/14 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
100%|██████████| 14/14 [00:00<00:00, 74.60it/s]


## Y_04

In [ ]:
train_Y_04.columns

Index(['X_01', 'X_02', 'X_03', 'X_05', 'X_06', 'X_07', 'X_08', 'X_09', 'X_10',
       'X_11', 'X_12', 'X_13', 'X_14', 'X_15', 'X_16', 'X_17', 'X_18', 'X_19',
       'X_20', 'X_21', 'X_22', 'X_24', 'X_25', 'X_26', 'X_27', 'X_28', 'X_29',
       'X_30', 'X_31', 'X_32', 'X_33', 'X_34', 'X_35', 'X_36', 'X_37', 'X_38',
       'X_39', 'X_40', 'X_41', 'X_42', 'X_43', 'X_44', 'X_45', 'X_46', 'X_49',
       'X_50', 'X_51', 'X_52', 'X_53', 'X_54', 'X_55', 'X_56', 'Y_04'],
      dtype='object')

In [ ]:
X = train_Y_04[['X_01', 'X_02', 'X_03', 'X_05', 'X_06', 'X_07', 'X_08', 'X_09', 'X_10',
       'X_11', 'X_12', 'X_13', 'X_14', 'X_15', 'X_16', 'X_17', 'X_18', 'X_19',
       'X_20', 'X_21', 'X_22', 'X_24', 'X_25', 'X_26', 'X_27', 'X_28', 'X_29',
       'X_30', 'X_31', 'X_32', 'X_33', 'X_34', 'X_35', 'X_36', 'X_37', 'X_38',
       'X_39', 'X_40', 'X_41', 'X_42', 'X_43', 'X_44', 'X_45', 'X_46', 'X_49',
       'X_50', 'X_51', 'X_52', 'X_53', 'X_54', 'X_55', 'X_56']]
y = train_Y_04.loc[:, 'Y_04':]

In [ ]:
target = test[X.columns]

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
kf = KFold(n_splits = 5, random_state = 42, shuffle = True)

In [ ]:
def objective_cat(trial):
    """
    Objective function to tune a `CatBoostRegressor` model.
    """

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    params = {
        'objective' :  'RMSE',
        'iterations':trial.suggest_int("iterations", 30, 400),
        'od_wait':trial.suggest_int('od_wait', 500, 2300),
        'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
        'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
        # 'subsample': trial.suggest_uniform('subsample',0,1),
        'random_strength': trial.suggest_uniform('random_strength',10,50),
        'depth': trial.suggest_int('depth',1, 15),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),
        'verbose' : False
    }


    model = CatBoostRegressor(
        # objective = 'MultiRMSE',
        random_state=42,
        **params,
    )
    model.fit(x_train, y_train, eval_set = [(x_test, y_test)], early_stopping_rounds = 500, verbose = 2000)

    pred = model.predict(x_test)

    rmse = mean_squared_error(y_test, pred, squared=False)


    return rmse

In [ ]:
study_catboost = optuna.create_study(direction="minimize")
study_catboost.optimize(objective_cat, n_trials=30)

[I 2022-08-05 01:43:26,970] A new study created in memory with name: no-name-5a17e587-326c-47b8-8025-15c8c02f253e


0:	learn: 2.6755814	test: 2.6456691	best: 2.6456691 (0)	total: 76.7ms	remaining: 27.7s


[I 2022-08-05 01:43:33,888] Trial 0 finished with value: 2.6284699301140084 and parameters: {'iterations': 362, 'od_wait': 868, 'learning_rate': 0.3107505502071587, 'reg_lambda': 69.5968038359678, 'random_strength': 37.239932622654244, 'depth': 8, 'min_data_in_leaf': 5, 'leaf_estimation_iterations': 8}. Best is trial 0 with value: 2.6284699301140084.


361:	learn: 2.3428037	test: 2.6880094	best: 2.6284699 (40)	total: 6.8s	remaining: 0us

bestTest = 2.628469938
bestIteration = 40

Shrink model to first 41 iterations.
0:	learn: 2.6901293	test: 2.6577827	best: 2.6577827 (0)	total: 17.5ms	remaining: 962ms


[I 2022-08-05 01:43:35,023] Trial 1 finished with value: 2.6288873274971576 and parameters: {'iterations': 56, 'od_wait': 2278, 'learning_rate': 0.06907441515923798, 'reg_lambda': 29.176790343551495, 'random_strength': 41.26242400833928, 'depth': 5, 'min_data_in_leaf': 25, 'leaf_estimation_iterations': 12}. Best is trial 0 with value: 2.6284699301140084.


...

102:	learn: 2.5628497	test: 2.6331824	best: 2.6257422 (55)	total: 1.82s	remaining: 0us

bestTest = 2.625742226
bestIteration = 55

Shrink model to first 56 iterations.
0:	learn: 2.6781077	test: 2.6479853	best: 2.6479853 (0)	total: 14.5ms	remaining: 2.06s


[I 2022-08-05 01:50:09,643] Trial 28 finished with value: 2.6271035017995685 and parameters: {'iterations': 143, 'od_wait': 1626, 'learning_rate': 0.2758231385823604, 'reg_lambda': 70.39848339985582, 'random_strength': 28.39704288166949, 'depth': 6, 'min_data_in_leaf': 17, 'leaf_estimation_iterations': 7}. Best is trial 16 with value: 2.625578899503457.


142:	learn: 2.5783735	test: 2.6373331	best: 2.6271035 (58)	total: 1.86s	remaining: 0us

bestTest = 2.627103509
bestIteration = 58

Shrink model to first 59 iterations.
0:	learn: 2.6740058	test: 2.6444104	best: 2.6444104 (0)	total: 15.7ms	remaining: 3.06s


[I 2022-08-05 01:50:12,607] Trial 29 finished with value: 2.6266068994904086 and parameters: {'iterations': 196, 'od_wait': 1025, 'learning_rate': 0.3495832914683483, 'reg_lambda': 71.19729468607613, 'random_strength': 32.342734114463894, 'depth': 8, 'min_data_in_leaf': 17, 'leaf_estimation_iterations': 5}. Best is trial 16 with value: 2.625578899503457.


195:	learn: 2.4794855	test: 2.6512959	best: 2.6266069 (43)	total: 2.85s	remaining: 0us

bestTest = 2.626606907
bestIteration = 43

Shrink model to first 44 iterations.


In [ ]:
params=study_catboost.best_params 
params['objective'] = 'RMSE'
params['random_state'] = 42 

In [ ]:
params

{'depth': 9,
 'iterations': 103,
 'leaf_estimation_iterations': 3,
 'learning_rate': 0.19357260131405404,
 'min_data_in_leaf': 1,
 'objective': 'RMSE',
 'od_wait': 1609,
 'random_state': 42,
 'random_strength': 18.108905551642163,
 'reg_lambda': 68.25673693779959}

In [ ]:
NRMSES = []
cb_pred = np.zeros(target.shape[0])
for i, idx in enumerate(kf.split(X, y)) :
    tr_x, tr_y = X.loc[idx[0]], y.loc[idx[0]]
    val_x, val_y = X.loc[idx[1]], y.loc[idx[1]]
    
    cb = CatBoostRegressor(**params)
    cb.fit(tr_x, tr_y, eval_set = [(tr_x, tr_y), (val_x, val_y)], early_stopping_rounds = 500, verbose = 2000)
    
    val_pred = cb.predict(val_x)
    NRMSE = mean_squared_error(val_y, val_pred, squared=False)
    print(f"{i + 1} Fold RMSE = {NRMSE}")
    NRMSES.append(NRMSE)
    
    fold_pred = cb.predict(target) / kf.n_splits
    cb_pred += fold_pred

0:	learn: 2.6369422	test: 2.6369422	test1: 2.7876289	best: 2.7876289 (0)	total: 16.4ms	remaining: 1.67s
102:	learn: 2.5731457	test: 2.5731457	test1: 2.7686963	best: 2.7663827 (68)	total: 1.63s	remaining: 0us

bestTest = 2.766382658
bestIteration = 68

Shrink model to first 69 iterations.
1 Fold NRMSE = 2.7663826552021606
0:	learn: 2.6790050	test: 2.6790050	test1: 2.6632261	best: 2.6632261 (0)	total: 18.4ms	remaining: 1.87s
102:	learn: 2.6234004	test: 2.6234004	test1: 2.6426479	best: 2.6414279 (26)	total: 1.63s	remaining: 0us

bestTest = 2.641427926
bestIteration = 26

Shrink model to first 27 iterations.
2 Fold NRMSE = 2.6414279318573146
0:	learn: 2.6970783	test: 2.6970783	test1: 2.6087794	best: 2.6087794 (0)	total: 19.8ms	remaining: 2.02s
102:	learn: 2.6381011	test: 2.6381011	test1: 2.5855843	best: 2.5835383 (29)	total: 1.76s	remaining: 0us

bestTest = 2.583538349
bestIteration = 29

Shrink model to first 30 iterations.
3 Fold NRMSE = 2.583538354984123
0:	learn: 2.6878771	test: 2.6878

In [ ]:
np.mean(NRMSES)

2.6504443969311007

In [ ]:
NRMSES

[2.7663826552021606, 2.6414279318573146, 2.583538354984123, 2.6104286456808037]